# Quarterly Campaign Disclosures for General 2020 Candidate Committees

This notebook includes a semi-automated walk-through of wrangling campaign finance data disclosed by committees of candidates for state and local offices in Missouri's General election.

## Downloads (manual)

To update our copy of the data, we first have to manually download a couple of files from the Missouri Ethics Commission's website ([mec.mo.gov](https://mec.mo.gov/)):

1. Report Summaries
  - Go to [`https://www.mec.mo.gov/MEC/Campaign_Finance/CF_ContrCSV.aspx`](https://www.mec.mo.gov/MEC/Campaign_Finance/CF_ContrCSV.aspx)
  - For Report Type, select **Report Summary**
  - For Year, select **2020**
  - Click the "Export to CSV" button
  - Save the file in the `data/` directory of this repository with the file name `report-summaries-2020.csv`
2. Candidates
  - Go to [`https://mec.mo.gov/MEC/Campaign_Finance/CF12_SearchElection.aspx`](https://mec.mo.gov/MEC/Campaign_Finance/CF12_SearchElection.aspx)
  - For Election Year, select **2020**
  - For Election Date, select **11/03/2020**
  - Save the file in the `data/` directory of this repository with the file name `candidates-2020-11-03.xls`
  
Once you've downloaded these files, you can re-run all the cells in the notebook to get an update version of the processed data.  

## Dependencies

In [1]:
import pandas as pd

## Report summaries

When we read data into a new dataframe, pandas by default will infer each column's data type. This default behavior often works well enough, especially for an initial exploration of the data.

However, we can also explicitly specify the data types for any column when reading data into a new dataframe, and this extra step has a couple of benefits:

- We get the exact data types—with the exact methods and attributes—that we want.
- We gain some non-trival efficiency when reading into a dataframe, especially from really large source files.

To specify each data type, we can define a `dict` that maps the column labels pandas [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes):

In [2]:
report_summary_data_types = {
    "MECID": "string",
    "Committee Name": "string",
    "Report": "string",
    "Report Year": "Int64",
    "Report Type": "string",
    "Previous Receipts": "float",
    "Contributions Received": "float",
    "Loans Received": "float",
    " Misc. Receipts": "float",
    " Receipts Subtotal": "float",
    "In-Kind Contributions": "float",
    " Total Receipts This Election": "float",
    "Previous Expenditures": "float",
    "Cash or Check Expenditures": "float",
    "In-Kind Expenditures": "float",
    "Credit Expenditures": "float",
    "Expenditure Subtotal": "float",
    "Total Expenditures": "float",
    "Previous Contributions": "float",
    "Cash/Check Contributions": "float",
    "Credit Contributions": "float",
    "In-Kind Contributions1": "float",
    "Contribution Subtotal": "float",
    "Total Contributions": "float",
    "Loan Disbursements": "float",
    "Disbursements Payments": "float",
    "Misc. Disbursements": "float",
    "Total Disbursements": "float",
    "Starting Money on Hand": "float",
    " Monetary Receipts": "float",
    "Check Disbursements": "float",
    "Cash Disbursements": "float",
    "Total Monetary Disbursements": "float",
    "Ending Money on Hand": "float",
    "Outstanding Indebtedness": "float",
    "Loans Recieved": "float",
    "New Expenditures": "float",
    "New Contributions": "float",
    "Payments Made on Loan": "float",
    "Debt Forgiven on Loans": "float",
    "Total Indebtendness": "float",
}

Then we can read in this csv file while passing in our dtype mapping.

In [3]:
report_summaries = pd.read_csv("data/report-summaries-2020.csv", dtype=report_summary_data_types)

The provided column headers are not optimal for use in Python. So let's fix them!

In [4]:
report_summaries.columns = report_summaries.columns \
    .str.strip() \
    .str.lower() \
    .str.replace(' ', '_') \
    .str.replace('-', '_') \
    .str.replace('.', '') \
    .str.replace('/', '_')  

Now let's look at the full list of columns.

In [5]:
report_summaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13924 entries, 0 to 13923
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   mecid                         13924 non-null  string 
 1   committee_name                13924 non-null  string 
 2   report                        13924 non-null  string 
 3   report_year                   13924 non-null  Int64  
 4   report_type                   13924 non-null  string 
 5   previous_receipts             10543 non-null  float64
 6   contributions_received        10543 non-null  float64
 7   loans_received                10543 non-null  float64
 8   misc_receipts                 10543 non-null  float64
 9   receipts_subtotal             10543 non-null  float64
 10  in_kind_contributions         10543 non-null  float64
 11  total_receipts_this_election  10543 non-null  float64
 12  previous_expenditures         10543 non-null  float64
 13  c

Next, let's create a new dataframe by filtering down to just the quarterly reports for 2020. In this step, we will also reset the index and drop the index from the original dataframe and drop the `committee_name` column, which will be redundant once we merge the report summaries with the candidates.

In [6]:
quarterly_report_summaries = report_summaries[report_summaries.report.str.contains('Quarterly', case=False)] \
    .reset_index(drop=True) \
    .drop(columns=['committee_name',])

It would be useful to be able to sort these report summaries by month. To allow that, we'll create a `month_num` column derived from the `report` column.

First, we need a function for getting the month_num based on the report name. Here's a really simple one:

In [7]:
def get_month_num(report):
    if 'January' in report:
        return 1
    elif 'April' in report:
        return 4
    elif 'July' in report:
        return 7
    elif 'October' in report:
        return 10

Then we apply this function to every value in the report column in order to get the value for the new column.

In [8]:
quarterly_report_summaries['month_num'] = quarterly_report_summaries.report.apply(get_month_num)

In [9]:
quarterly_report_summaries.month_num.value_counts()

7     2817
4     2685
1     2300
10    2285
Name: month_num, dtype: int64

In [10]:
len(quarterly_report_summaries)

10087

In [11]:
2817 + 2685 + 2300 + 1932

9734

## Candidates

Just as we were when reading in the report summaries, let's specify our dtypes

In [12]:
candidate_data_types = {
    'MECID': "string",
    'Committee Name': "string",
    'Candidate Name': "string",
    # categorical?
    'Party': "string",
    # categorical?
    'Office Sought': "string",
    # categorical?
    'Status': "string",
}

Can't get pandas to read this source as an Excel file, but it does look and parse like an HTML table.

In [13]:
candidates = pd.read_html('data/candidates-2020-11-03.xls')[0] \
    .astype(candidate_data_types) \
    .fillna("")

Again, fix the columns:

In [14]:
candidates.columns = candidates.columns \
    .str.strip() \
    .str.lower() \
    .str.replace(" ", "_")

In [15]:
candidates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   mecid           680 non-null    string
 1   committee_name  680 non-null    string
 2   candidate_name  680 non-null    string
 3   party           680 non-null    string
 4   office_sought   680 non-null    string
 5   status          680 non-null    string
dtypes: string(6)
memory usage: 32.0 KB


We can use the `office_sought` to filter down to the get the exact candidates we want. But first, we have to figure get all the unique values of this column:

In [16]:
sorted(candidates.office_sought.unique())

['Assessor - Bates County',
 'Assessor - Boone County',
 'Assessor - Buchanan County',
 'Assessor - Cass County',
 'Assessor - Clark County',
 'Assessor - Clay County',
 'Assessor - Cole County',
 'Assessor - Crawford County',
 'Assessor - Daviess County',
 'Assessor - Dent County',
 'Assessor - Franklin County',
 'Assessor - Greene County',
 'Assessor - Jefferson County',
 'Assessor - Marion County',
 'Assessor - Moniteau County',
 'Assessor - Monroe County',
 'Assessor - Ozark County',
 'Assessor - Pettis County',
 'Assessor - Pike County',
 'Assessor - Platte County',
 'Assessor - Ray County',
 'Assessor - Scott County',
 'Assessor - St. Francois County',
 'Assessor - Warren County',
 'Assessor - Washington County',
 'Associate Circuit Judge - Cole County',
 'Associate Circuit Judge - St. Charles County',
 'Associate Commissioner - Adair County',
 'Associate Commissioner - Barry County',
 'Associate Commissioner - Bates County',
 'Associate Commissioner - Boone County',
 'Associate 

Here are the offices we care about.

In [17]:
target_offices = [
    "State Senator - District 19 - Missouri State Senate",
    "State Representative - District 44 - Missouri House of Representatives",
    "State Representative - District 47 - Missouri House of Representatives",
    "State Representative - District 50 - Missouri House of Representatives",
    "Secretary of State - Missouri - Office of the Secretary of State",
    "Secretary of State - Office of the Secretary of State",
    "Governor - Office of the Governor",
    "Governor - State of Missouri - Office of the Governor",
    "State Treasurer - Office of the State Treasurer",
    "Lieutenant Governor",
    "Lieutenant Governor - Office of the Lieutenant Governor",
    "Associate Commissioner - Boone County",
    "Associate Commissioner - District 1 (Southern) - Boone County",
    "Associate Commissioner - District 2 - Boone County",
    "Sheriff - Boone County",
]

We can use this list to set up a filter.

In [18]:
office_filter = candidates['office_sought'].isin(target_offices)

And then use that filter, along with another filter for "active" candidates, to get the exact subset of candidates we care about.

In [19]:
target_candidates = candidates[candidates.status.eq('A') & office_filter] \
    .reset_index(drop=True) \
    .drop(columns=['status',])

Also going to add another column here to facilite grouping by office type. This will be derived by splitting the `office_sought` values at the ` - ` substring and taking the first value in the resulting list.

In [20]:
target_candidates['office_type'] = target_candidates.office_sought.str.split(' - ', expand=True)[0]

## Merging

The `target_candidates` dataframe has exactly the candidates we want, but `quarterly_reports` still includes non-candidate committees.

In [21]:
candidate_report_summaries_all_quarters = target_candidates \
    .merge(quarterly_report_summaries, on='mecid', validate='one_to_many') \
    .reset_index(drop=True) \
    .sort_values(['office_sought', 'candidate_name', 'month_num'])

## Who's missing?

Here are the people who haven't filed yet.

In [22]:
oct_report_summaries = quarterly_report_summaries[quarterly_report_summaries.month_num.eq(10)]

In [23]:
candidate_report_summaries_oct = target_candidates \
    .merge(oct_report_summaries, on='mecid', validate='one_to_many', how="left") \
    .reset_index(drop=True) \
    .sort_values(['office_sought', 'candidate_name', 'month_num'])

In [24]:
missing_oct_reports = candidate_report_summaries_oct[candidate_report_summaries_oct.report.isna()]

In [25]:
missing_oct_reports.iloc[:,:6]

,mecid,committee_name,candidate_name,party,office_sought,office_type
25,C201232E,Jerome H. Bauer,Jerome Bauer,G,Governor - Office of the Governor,Governor
32,C201322E,Paul T. Lehmann,Paul Lehmann,G,Secretary of State - Office of the Secretary o...,Secretary of State
20,C111215,Friends Of Caleb Rowden,CALEB ROWDEN,R,State Senator - District 19 - Missouri State S...,State Senator


## Outputting

In [27]:
candidate_report_summaries_all_quarters \
    .to_csv("candidate-committees-filings-post-oct-q.csv", index=False)